<a href="https://colab.research.google.com/github/simon-mellergaard/GAI-with-LLMs/blob/main/Project%20codes/question1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project

This is the answers to the questions in the project for the course

## Question 1

[Insert question here]

In [1]:
bar = "foo"
print(bar)

foo
